## Analyze A/B Test Results


## Table of Contents
- [Introduction](#intro)
- [Part I - Probability](#probability)
- [Part II - A/B Test](#ab_test)
- [Part III - Regression](#regression)


<a id='intro'></a>


<a id='probability'></a>
#### Part I - Probability

To get started, let's import our libraries.

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
random.seed(42)




a. Read in the dataset and take a look at the top few rows here:

In [2]:
df = pd.read_csv('ab_data.csv')
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


b. find the number of rows in the dataset.

In [3]:
##df rows and coloumns
df.shape

(294478, 5)

c. The number of unique users in the dataset.

In [4]:
df.nunique()

user_id         290584
timestamp       294478
group                2
landing_page         2
converted            2
dtype: int64

d. The proportion of users converted.

In [5]:
df.converted.mean()

0.11965919355605512

e. The number of times the `new_page` and `treatment` don't match.

In [6]:
#The number of times the new_page and treatment don't line up.


dont_match = df[((df['group'] == 'treatment') == (df['landing_page'] == 'new_page')) == False]
print("The number of times the new_page and treatment don't match is " +str(dont_match.shape[0]))

The number of times the new_page and treatment don't match is 3893


f. Do any of the rows have missing values?

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
user_id         294478 non-null int64
timestamp       294478 non-null object
group           294478 non-null object
landing_page    294478 non-null object
converted       294478 non-null int64
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


`2.` For the rows where **treatment** does not match with **new_page** or **control** does not match with **old_page**, we cannot be sure if this row truly received the new or old page. 

In [8]:
#TN For Treatment & New Page , CO for control & old Page
TN = df[(df.group == 'treatment') & (df.landing_page == 'new_page')] 
CO = df[(df.group == 'control') & (df.landing_page == 'old_page')] 
DF_CON= [TN, CO]
df2 = pd.concat(DF_CON)

In [9]:
# Double Check all of the correct rows were removed - this should be 0
df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].shape[0]

0

a. How many unique **user_id**s are in **df2**?

In [10]:
df2.user_id.nunique()

290584

b. There is one **user_id** repeated in **df2**.  What is it?

In [11]:
df2[df2['user_id'].duplicated()]

,user_id,timestamp,group,landing_page,converted
2893,773192,2017-01-14 02:55:59.590927,treatment,new_page,0


c. What is the row information for the repeat **user_id**? 

In [12]:
df2[df2['user_id']==773192]

,user_id,timestamp,group,landing_page,converted
1899,773192,2017-01-09 05:37:58.781806,treatment,new_page,0
2893,773192,2017-01-14 02:55:59.590927,treatment,new_page,0


d. Remove **one** of the rows with a duplicate **user_id**, but keep your dataframe as **df2**.

In [13]:
df2.drop([2893] , inplace = True , axis = 0)

In [14]:
df2.converted.mean()

0.11959708724499628

b. Given that an individual was in the `control` group, what is the probability they converted?

In [15]:
df2[df2['group']=='control'].converted.mean()

0.1203863045004612

c. Given that an individual was in the `treatment` group, what is the probability they converted?

In [16]:
df2[df2['group']=='treatment'].converted.mean()

0.11880806551510564

d. What is the probability that an individual received the new page?

In [17]:
(df2['landing_page']=='new_page').mean()

0.50006194422266881

e. Consider your results from parts (a) through (d) above, and explain below whether you think there is sufficient evidence to conclude that the new treatment page leads to more conversions.

theirs no sufficient evidence tell us that we must start applying the new page , the conversion probability is almost the same between the old page and the new page , the conversion rate of the old page is a little bit larger than the old page .  

<a id='ab_test'></a>
### Part II - A/B Test

Notice that because of the time stamp associated with each event, you could technically run a hypothesis test continuously as each observation was observed.  

However, then the hard question is do you stop as soon as one page is considered significantly better than another or does it need to happen consistently for a certain amount of time?  How long do you run to render a decision that neither page is better than another?  

These questions are the difficult parts associated with A/B tests in general.  


`1.` For now, consider you need to make the decision just based on all the data provided.  If you want to assume that the old page is better unless the new page proves to be definitely better at a Type I error rate of 5%, what should your null and alternative hypotheses be?  You can state your hypothesis in terms of words or in terms of **$p_{old}$** and **$p_{new}$**, which are the converted rates for the old and new pages.



- H_0 : Pnew - Pold <=0
- H_1 : Pnew - Pold > 0


`2.` Assume under the null hypothesis, $p_{new}$ and $p_{old}$ both have "true" success rates equal to the **converted** success rate regardless of page - that is $p_{new}$ and $p_{old}$ are equal. Furthermore, assume they are equal to the **converted** rate in **ab_data.csv** regardless of the page. <br><br>

Use a sample size for each page equal to the ones in **ab_data.csv**.  <br><br>

Perform the sampling distribution for the difference in **converted** between the two pages over 10,000 iterations of calculating an estimate from the null.  <br><br>

Use the cells below to provide the necessary parts of this simulation.  If this doesn't make complete sense right now, don't worry - you are going to work through the problems below to complete this problem.  You can use **Quiz 5** in the classroom to make sure you are on the right track.<br><br>


a. What is the **conversion rate** for $p_{new}$ under the null? 
- H0 = Pnew - Pold <=0
- H1 = Pnew - Pold >0

In [18]:
Pnew = df2.converted.mean()
Pnew

0.11959708724499628

b. What is the **conversion rate** for $p_{old}$ under the null? <br><br>

In [19]:
Pold = df2.converted.mean()
Pold

0.11959708724499628

c. What is $n_{new}$, the number of individuals in the treatment group?

In [20]:
Nnew = len(df2[df2['group']=='treatment'])
Nnew

145310

d. What is $n_{old}$, the number of individuals in the control group?

In [21]:
Nold = len(df2[df2['group']=='control'])
Nold

145274

e. Simulate $n_{new}$ transactions with a conversion rate of $p_{new}$ under the null.  Store these $n_{new}$ 1's and 0's in **new_page_converted**.

In [22]:
np_convert = np.random.choice([1, 0], size=Nnew, p=[Pnew, (1-Pnew)]).mean()
np_convert

0.11949624939783911

f. Simulate $n_{old}$ transactions with a conversion rate of $p_{old}$ under the null.  Store these $n_{old}$ 1's and 0's in **old_page_converted**.

In [23]:
op_convert = np.random.choice([1, 0], size=Nold, p=[Pold, (1-Pold)]).mean()
op_convert

0.11966353235954127

g. Find $p_{new}$ - $p_{old}$ for your simulated values from part (e) and (f).

In [24]:
convert_differance = np_convert-op_convert
convert_differance

-0.00016728296170216705

h. Create 10,000 $p_{new}$ - $p_{old}$ values using the same simulation process you used in parts (a) through (g) above. Store all 10,000 values in a NumPy array called **p_diffs**.

In [ ]:
p_diffs = []
for _ in range(10000):
    np_convert = np.random.choice([1, 0], size=Nnew, p=[Pnew, (1-Pnew)]).mean()
    op_convert = np.random.choice([1, 0], size=Nold, p=[Pold, (1-Pold)]).mean()
    differance = np_convert - op_convert
    p_diffs.append(differance)


i. Plot a histogram of the **p_diffs**.  Does this plot look like what you expected?  Use the matching problem in the classroom to assure you fully understand what was computed here.

In [ ]:
plt.hist(p_diffs);

j. What proportion of the **p_diffs** are greater than the actual difference observed in **ab_data.csv**?

In [40]:
K_diff = df2[df2['group']=='treatment'].converted.mean() - df2[df2['group']=='control'].converted.mean() 

In [42]:
(p_diffs >= K_diff).mean()

0.89659999999999995

k. Please explain using the vocabulary you've learned in this course what you just computed in part **j.**  What is this value called in scientific studies?  What does this value mean in terms of whether or not there is a difference between the new and old pages?

Here The P-Value Is Large! it is 0.897 , so we can't reject the null hypothesis w should accept it . 
thats means that the new page is doing not better than the old page!! , the old page is doing well more than the new page .

l. We could also use a built-in to achieve similar results.  Though using the built-in might be easier to code, the above portions are a walkthrough of the ideas that are critical to correctly thinking about statistical significance. Fill in the below to calculate the number of conversions for each page, as well as the number of individuals who received each page. Let `n_old` and `n_new` refer the the number of rows associated with the old page and new pages, respectively.

In [43]:
import statsmodels.api as sm

convert_old = sum(df2.query("group == 'control'")['converted'])
convert_new = sum(df2.query("group == 'treatment'")['converted'])
n_old = len(df2.query('landing_page=="old_page"')) #rows_associated with old_page
n_new = len(df2.query('landing_page=="new_page"')) #rows associated with new_page


/opt/conda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


m. Now use `stats.proportions_ztest` to compute your test statistic and p-value.  [Here](https://docs.w3cub.com/statsmodels/generated/statsmodels.stats.proportion.proportions_ztest/) is a helpful link on using the built in.

In [44]:
Z_score, P_value = sm.stats.proportions_ztest([convert_new, convert_old], [n_new, n_old], alternative='larger')
Z_score, P_value

(-1.3109241984234394, 0.90505831275902449)

n. What do the z-score and p-value you computed in the previous question mean for the conversion rates of the old and new pages?  Do they agree with the findings in parts **j.** and **k.**?

Z-score is smaller than value of confidence 95% , We fail to reject the null hypothesis , the old page converted more users than the new page.

<a id='regression'></a>
### Part III - A regression approach

`1.` In this final part, you will see that the result you achieved in the A/B test in Part II above can also be achieved by performing regression.<br><br> 

a. Since each row is either a conversion or no conversion, what type of regression should you be performing in this case?

We Will use Logistic Regression cause we deal with categorical data

b. The goal is to use **statsmodels** to fit the regression model you specified in part **a.** to see if there is a significant difference in conversion based on which page a customer receives. However, you first need to create in df2 a column for the intercept, and create a dummy variable column for which page each user received.  Add an **intercept** column, as well as an **ab_page** column, which is 1 when an individual receives the **treatment** and 0 if **control**.

In [45]:
df2['intercept']=1
df2[['cont','ab_page']] = pd.get_dummies(df2['group'])
df2.sample(10)

,user_id,timestamp,group,landing_page,converted,intercept,cont,ab_page
151431,906507,2017-01-23 01:45:09.971542,treatment,new_page,0,1,0,1
24928,922066,2017-01-05 17:25:40.185331,control,old_page,0,1,1,0
290615,696605,2017-01-17 12:17:56.708738,treatment,new_page,0,1,0,1
169126,887712,2017-01-04 14:44:42.976142,treatment,new_page,0,1,0,1
152933,882200,2017-01-16 21:59:58.118909,treatment,new_page,0,1,0,1
226220,926189,2017-01-22 11:59:50.583191,control,old_page,0,1,1,0
108488,720625,2017-01-19 10:49:45.240856,treatment,new_page,0,1,0,1
27831,926352,2017-01-03 21:47:24.076866,treatment,new_page,0,1,0,1
102736,748505,2017-01-06 11:54:23.463873,treatment,new_page,0,1,0,1
283268,844111,2017-01-13 19:15:51.279280,control,old_page,1,1,1,0


In [46]:
df2.drop('cont' , axis = 1 ,inplace = True)


In [47]:
df2.head()

,user_id,timestamp,group,landing_page,converted,intercept,ab_page
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0,1,1
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0,1,1
6,679687,2017-01-19 03:26:46.940749,treatment,new_page,1,1,1
8,817355,2017-01-04 17:58:08.979471,treatment,new_page,1,1,1
9,839785,2017-01-15 18:11:06.610965,treatment,new_page,1,1,1


c. Use **statsmodels** to instantiate your regression model on the two columns you created in part b., then fit the model using the two columns you created in part **b.** to predict whether or not an individual converts. 

In [48]:
logic_reg = sm.Logit(df2['converted'], df2[['intercept', 'ab_page']])
results = logic_reg.fit()

Optimization terminated successfully.
         Current function value: 0.366118
         Iterations 6


d. Provide the summary of your model below, and use it as necessary to answer the following questions.

In [49]:
results.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                          Results: Logit
==================================================================
Model:              Logit            No. Iterations:   6.0000     
Dependent Variable: converted        Pseudo R-squared: 0.000      
Date:               2021-03-20 18:42 AIC:              212780.3502
No. Observations:   290584           BIC:              212801.5095
Df Model:           1                Log-Likelihood:   -1.0639e+05
Df Residuals:       290582           LL-Null:          -1.0639e+05
Converged:          1.0000           Scale:            1.0000     
-------------------------------------------------------------------
              Coef.   Std.Err.      z      P>|z|    [0.025   0.975]
-------------------------------------------------------------------
intercept    -1.9888    0.0081  -246.6690  0.0000  -2.0046  -1.9730
ab_page      -0.0150    0.0114    -1.3109  0.1899  -0.0374   0.0074
==================================================================

"""

e. What is the p-value associated with **ab_page**? Why does it differ from the value you found in **Part II**?<br><br>  **Hint**: What are the null and alternative hypotheses associated with your regression model, and how do they compare to the null and alternative hypotheses in **Part II**?

 The P-value with ap_page is 0.18,  0.18 is bigger then 0.05 our alpha, so we still can't reject our null hypothesis.

f. Now, you are considering other things that might influence whether or not an individual converts.  Discuss why it is a good idea to consider other factors to add into your regression model.  Are there any disadvantages to adding additional terms into your regression model?

adding more factors maybe lead to miss accuracy , also will increase or decrease the confidence intervals ,
one of the disadvantage of the logistic reggerasion of adding addtional terms it may reduce the power of the analysis

g. Now along with testing if the conversion rate changes for different pages, also add an effect based on which country a user lives in. You will need to read in the **countries.csv** dataset and merge together your datasets on the appropriate rows.  [Here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.join.html) are the docs for joining tables. 

Does it appear that country had an impact on conversion?  Don't forget to create dummy variables for these country columns - **Hint: You will need two columns for the three dummy variables.** Provide the statistical output as well as a written response to answer this question.

In [50]:
df3 = pd.read_csv('countries.csv')
df3.head()
df_new = df3.set_index('user_id').join(df2.set_index('user_id'), how='inner')
df_new.head()

,country,timestamp,group,landing_page,converted,intercept,ab_page
user_id,,,,,,,
834778,UK,2017-01-14 23:08:43.304998,control,old_page,0,1,0
928468,US,2017-01-23 14:44:16.387854,treatment,new_page,0,1,1
822059,UK,2017-01-16 14:04:14.719771,treatment,new_page,1,1,1
711597,UK,2017-01-22 03:14:24.763511,control,old_page,0,1,0
710616,UK,2017-01-16 13:14:44.000513,treatment,new_page,0,1,1


In [51]:
df_new.country.unique()

array(['UK', 'US', 'CA'], dtype=object)

In [52]:
df_new[['ca', 'uk', 'us']] = pd.get_dummies(df_new['country'])
df_new.head()

,country,timestamp,group,landing_page,converted,intercept,ab_page,ca,uk,us
user_id,,,,,,,,,,
834778,UK,2017-01-14 23:08:43.304998,control,old_page,0,1,0,0,1,0
928468,US,2017-01-23 14:44:16.387854,treatment,new_page,0,1,1,0,0,1
822059,UK,2017-01-16 14:04:14.719771,treatment,new_page,1,1,1,0,1,0
711597,UK,2017-01-22 03:14:24.763511,control,old_page,0,1,0,0,1,0
710616,UK,2017-01-16 13:14:44.000513,treatment,new_page,0,1,1,0,1,0


In [53]:
df_new.drop('ca' , axis = 1 , inplace = True)

In [54]:
logic_reg = sm.Logit(df_new['converted'], df_new[['intercept', 'uk', 'us']])
result = logic_reg.fit()
result.summary2()

Optimization terminated successfully.
         Current function value: 0.366116
         Iterations 6


<class 'statsmodels.iolib.summary2.Summary'>
"""
                          Results: Logit
==================================================================
Model:              Logit            No. Iterations:   6.0000     
Dependent Variable: converted        Pseudo R-squared: 0.000      
Date:               2021-03-20 18:42 AIC:              212780.8333
No. Observations:   290584           BIC:              212812.5723
Df Model:           2                Log-Likelihood:   -1.0639e+05
Df Residuals:       290581           LL-Null:          -1.0639e+05
Converged:          1.0000           Scale:            1.0000     
-------------------------------------------------------------------
               Coef.   Std.Err.     z      P>|z|    [0.025   0.975]
-------------------------------------------------------------------
intercept     -2.0375    0.0260  -78.3639  0.0000  -2.0885  -1.9866
uk             0.0507    0.0284    1.7863  0.0740  -0.0049   0.1064
us             0.0408    0.0269    1.5178  0.1291  -0.0119   0.0935
==================================================================

"""

Answer: The p-values for the countries are bigger than 0.05, so also theirs no evidence to rejcet the null hypthesis , countries doesn't impact evivdence to reject the null!


h. Though you have now looked at the individual factors of country and page on conversion, we would now like to look at an interaction between page and country to see if there significant effects on conversion.  Create the necessary additional columns, and fit the new model.  

Provide the summary results, and your conclusions based on the results.

In [55]:
df_new['US_ab_page'] = df_new['us']*df_new['ab_page']

In [56]:
df_new['UK_ab_page'] = df_new['uk']*df_new['ab_page']
df_new.head()

,country,timestamp,group,landing_page,converted,intercept,ab_page,uk,us,US_ab_page,UK_ab_page
user_id,,,,,,,,,,,
834778,UK,2017-01-14 23:08:43.304998,control,old_page,0,1,0,1,0,0,0
928468,US,2017-01-23 14:44:16.387854,treatment,new_page,0,1,1,0,1,1,0
822059,UK,2017-01-16 14:04:14.719771,treatment,new_page,1,1,1,1,0,0,1
711597,UK,2017-01-22 03:14:24.763511,control,old_page,0,1,0,1,0,0,0
710616,UK,2017-01-16 13:14:44.000513,treatment,new_page,0,1,1,1,0,0,1


In [57]:
logic_reg = sm.Logit(df_new['converted'], df_new[['intercept', 'ab_page', 'us', 'uk', 'US_ab_page', 'UK_ab_page']])
result = logic_reg.fit()
result.summary2()

Optimization terminated successfully.
         Current function value: 0.366112
         Iterations 6


<class 'statsmodels.iolib.summary2.Summary'>
"""
                             Results: Logit
=========================================================================
Model:                 Logit              No. Iterations:     6.0000     
Dependent Variable:    converted          Pseudo R-squared:   0.000      
Date:                  2021-03-20 18:44   AIC:                212782.5603
No. Observations:      290584             BIC:                212835.4585
Df Model:              4                  Log-Likelihood:     -1.0639e+05
Df Residuals:          290579             LL-Null:            -1.0639e+05
Converged:             1.0000             Scale:              1.0000     
-------------------------------------------------------------------------
            Coef.    Std.Err.     z     P>|z|      [0.025       0.975]   
-------------------------------------------------------------------------
intercept  -2.0366      0.0280 -72.6176 0.0000       -2.0916      -1.9817
ab_page    -0.0018      0.0209  -0.0861 0.9313       -0.0427       0.0391
us          0.0501      0.0297   1.6912 0.0908       -0.0080       0.1083
uk          0.0507      0.0284   1.7860 0.0741       -0.0049       0.1064
US_ab_page -0.0094 704237.3061  -0.0000 1.0000 -1380279.7660 1380279.7472
US_ab_page -0.0094 704237.3061  -0.0000 1.0000 -1380279.7660 1380279.7472
=========================================================================

"""

Answer : all the p-values are greater than the aplha so agian and agian we can't rejcet the null hypothesis cause theirs no evidence to do than

In [58]:
np.exp(result.params)


intercept     0.130468
ab_page       0.998202
us            1.051427
uk            1.052017
US_ab_page    0.990662
US_ab_page    0.990662
dtype: float64

Conc : 
        we dont have the evidence to rejcet the null hypthesis and accept the alterntive hypothesis , from my analysis i would say the to e-commerce company to save their money , time  and power to make a new page and its allready doing like the old page no big differance added!! so stay with the old page.

In [77]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'Analyze_ab_test_results_notebook.ipynb'])

0